<a href="https://colab.research.google.com/github/KubaDavid/VSE-AI/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semestral work Keras and Tensor
This is the last semestral work from VŠE AI 4IZ231\
@authors Kotzias et. al (2015) and Zamazal (2020)\
@editor Jakub David


In [4]:
# Reading data

import pandas as pd

dfAmazon = pd.read_csv('amazon_cells_labelled.csv', names=['sentence', 'label'], sep='\t')

sentences = dfAmazon['sentence'].values

labels = dfAmazon['label'].values
labels = labels.astype(int)


# Checking the data format
print(sentences.shape)
print(labels.shape)
print("{}:{}".format(sentences[10], labels[10]))

(1000,)
(1000,)
And the sound quality is great.:1


In [5]:
# Dividing to train, test and validation datasets

from sklearn.model_selection import train_test_split

sentences_train, sentences_test, labels_train, labels_test = train_test_split(sentences, labels, test_size=0.25, random_state=1000)
sentences_train, sentences_val, labels_train, labels_val = train_test_split(sentences_train, labels_train, test_size=0.2, random_state=1000)

# Data check
print(sentences_train.shape)
print(labels_train.shape)
print("{}:{}".format(sentences_train[9], labels_train[9]))
print(sentences_val.shape)
print(labels_val.shape)
print("{}:{}".format(sentences_val[9], labels_val[9]))
print(sentences_test.shape)
print(labels_test.shape)
print("{}:{}".format(sentences_test[9], labels_test[9]))

(600,)
(600,)
It lasts less than 3o minutes, if I actually try to use the phone.My wife has the same phone with the same problem.:0
(150,)
(150,)
The cable looks so thin and flimsy, it is scary.:0
(250,)
(250,)
Disappointing accessory from a good manufacturer.:0


In [6]:
# Preprocessing into Bag of Words

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True)
vectorizer.fit(sentences_train)

# Check
print(vectorizer.vocabulary_['and'])
print(vectorizer.vocabulary_['good'])
print(vectorizer.vocabulary_['service'])
print(sentences_train[9])

X_train = vectorizer.transform(sentences_train)
X_val = vectorizer.transform(sentences_val)
X_test = vectorizer.transform(sentences_test)

# Check
print(X_train.shape)
print(X_train[9, :])
print(X_val.shape)
print(X_val[9, :])
print(X_test.shape)
print(X_test[9, :])

y_train = labels_train
y_val = labels_val
y_test = labels_test

65
518
1029
It lasts less than 3o minutes, if I actually try to use the phone.My wife has the same phone with the same problem.
(600, 1352)
  (0, 11)	1
  (0, 35)	1
  (0, 540)	1
  (0, 581)	1
  (0, 624)	1
  (0, 654)	1
  (0, 664)	1
  (0, 743)	1
  (0, 768)	1
  (0, 856)	1
  (0, 900)	1
  (0, 1001)	1
  (0, 1153)	1
  (0, 1156)	1
  (0, 1185)	1
  (0, 1212)	1
  (0, 1244)	1
  (0, 1312)	1
  (0, 1322)	1
(150, 1352)
  (0, 65)	1
  (0, 178)	1
  (0, 466)	1
  (0, 622)	1
  (0, 624)	1
  (0, 687)	1
  (0, 1080)	1
  (0, 1156)	1
(250, 1352)
  (0, 334)	1
  (0, 488)	1
  (0, 518)	1


In [ ]:
# Convolution network
from keras.models import Sequential
from keras import layers

# Number of attributes
input_dim = X_train.shape[1]
print(input_dim)

# Convolution network
convolutionModel = Sequential()
convolutionModel.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
convolutionModel.add(layers.Dense(1, activation='sigmoid'))
convolutionModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
convolutionModel.summary()

# Single layer network
singleModel = Sequential()
singleModel.add(layers.Dense(1, input_dim=input_dim, activation='sigmoid'))
singleModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
singleModel.summary()